# Data Analyst Project 3
Data Wrangle (Retrieve, Analyze and Clean) OpenStreetMaps Data from the City of Dresden

_by Benjamin Söllner, benjamin.soellner@gmail.com_

_based on the Udacity.com Data Wrangling With MongoDB_


<img src="city_dresden_json.png" alt="The city of Dresden as a JSON object illustration" width="400" height="312" style="display: inline; margin: 6pt;" />

##Abstract
This paper describes describes the process of downloading, analyzing and cleaning of an OpenStreet Map data set of my former home town as a student: [Dresden](https://en.wikipedia.org/wiki/Dresden), a state capital in eastern Germany, [a baroque town beautifully located on the board of the river Elbe](https://www.google.de/search?q=dresden&newwindow=1&es_sm=122&source=lnms&tbm=isch&sa=X&ved=0CAkQ_AUoA2oVChMIrc3d_cT3xwIVh2fbCh3Y6QYu&biw=1920&bih=969) and town home to a high-tech conglomerate from the micro-electronics sector called [Silicon Saxony](https://en.wikipedia.org/wiki/Silicon_Saxony).

In this paper, first, the pipeline (and python script) to perform retrieval, analysis and cleaning of the data is introduced (chapters [Approach](#Approach)) and results of the analysis stage are presented (chapters [Data Format](#Data-Format), [Descriptive Statistics](#Descriptive-Statistics), [Data Quality](#Data-Quality)). Especially during the Descriptive Statistics analysis, interesting facts of Dresden are uncovered, like the most popular religion, sport, beer, cuisine or leisure activity.

For the cleaning stage, canonicalizing phone numbers present in the data set was the challange of choice, since the originally proposed method of cleaning street names was not applicable for this data set (chapter [Cleaning Phone Numbers](#Cleaning-Phone-Numbers)). The paper is finally concluded with some further ideas (chapter [Conclusion and Further Ideas](#Conclusion-and-Further-Ideas)).

##The Approach
I implemented retrieving / storing / analysing and cleaning in a python script. The script can be called like so:
```
# python project.py
Usage:
  python project.py -d    Download & unpack bz2 file to OSM file (experimental)
  python project.py -p    Process OSM file and write JSON file
  python project.py -w    Write JSON file to MongoDB
  python project.py -f    Audit format / structure of data
  python project.py -s    Audit statistics of data
  python project.py -q    Audit quality of data
  python project.py -c    Clean data in MongoDB
  python project.py -C    Clean data debug mode - don't actually write to DB
```

Different options can be combined, so ``` python project.py -dpwfsqc ``` will do the whole round trip. During the process, I re-used most of the code and data format developed during the "Data Wrangling With MongoDB" Udacity course. For example, the data format used for storing the data (```-p``` and ```-w``` option) is completely based on [Lesson 6](https://www.udacity.com/course/viewer#!/c-ud032/l-768058569)  - with some fine-tuning.

Some output of the script is shown on the terminal, some is written to local files. If a file is written, this is indicated in the terminal output. A sample of the script's terminal output is included in the ```output_*.txt``` files included in the submission.

##Data Format
**Note:** Use ```python project.py -f``` to obtain the data for this chapter. This is a long-running process which might take a few hours to complete! There is an output file written to [```Project/data/audit_format_map.csv```](Project/data/audit_format_map.csv) which can be beautified into an [Excel spreadsheet](Project/data/audit_format_map.xlsx).

![A picture of the excel spreadsheet audit_format.xlsx](audit_format.png)

First, the data format was audited, which consisted of going through all the documents and aggregating the occurence of any attributes and the prevalence of their types (```str```ing, ```int```eger, ```float``` and ```other```). For this, batches of 1000 documents each are retrieved from the collection and each combed through by the python code while a Python Dataframe keeps track of the counters. Since there are 1,360,000 elements, this process takes many hours; an alternative would be to run the query natively in JavaScript code on the MongoDB shell or to issue the command as a BSON command.

The overview of the format showed no obvious big problems with the data at first glance but provided valuable insights for further analysis, especially to query [Descriptive Statistics](#Descriptive-Statistics). Some further interesting insights:

* One area of improvement could be the phone number, which is scattered across multiple data fields (```address:phone```, ```phone``` and ```phone_mobile```) and was identified as a potential candidate for cleaning (see [Auditing Phone Numbers](#Auditing-Phone-Numbers) and [Cleaning Phone Numbers](#Cleaning-Phone-Numbers)).
* Also, it is evident that some values are present in the dataset as sometimes ```str```ing, othertimes numeric: The XML parsing process takes care that each value is, whenever parsable, stored as integer or float. For attributes like street numbers, mixed occurences may be in the data set.
* This automatic parsing of ```int``` or ```float``` turned out to be not always useful: a problem are leading zeros which in certain cases hold semantics. For german phone numbers, a leading zero signifies the start of an area code (```0```) or the start of a country code (```00```). For german postcodes, a leading zero in a postcode represents the german state of Saxony. As an outcome of this insight, I changed the parsing routine of the XML data to only parse values as numeric, if they do not contain a leading zero (```not s.startswith("0")```)
* I checked some of the lesser-common values for sanity. E.g., there is a parameter ```dogshit``` which appears three times. As it turns out, this is not a prank of some map editors, who document dog feces they find in the area, but an indication about whether a public trash can contains a dispenser of plastic bags for relevant situations.

##Descriptive Statistics
**Note:** Use ```python project.py -s``` to obtain the data for this chapter. See Sample Output in file [```Project/output_project.py_-s.txt```](Project/output_project.py_-s.txt).

A couple of basic MongoDB queries were run to explore the data set based on the knowledge of its format from the previous chapter. The queries produce mostly rankings of values for certain data fields. Some of them are subsequently also visualized in a ggplot graph (png file) drawing on the skill set gained in [Udacity's Intro to Data Science course, Lesson 4: Data Visualization](https://www.udacity.com/course/viewer#!/c-ud359/l-692548568) while not too much effort was put in making the graphs look particularily beautiful. The graphs are located in ```Project/data/stats_*.png```.

###Users Involved
There were about 1634 users involved in creating the data set, the top 10 of all users accounts for 40% of the created data. There is no direct evidence from the user name that any of them are bot-like users. This could be determined by further research. Many users (over 60%) have made less than 10 entries.

In [10]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_users(...):
l = list(project_coll.aggregate([
            {"$match": {"created.user": {"$exists": True}}},
            {"$group": {"_id": "$created.user", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]))
print str(len(l)) + " users were involved:"
pprint.pprint(l[1:10]+["..."]+l[-10:])

1634 users were involved:
[{u'_id': u'Wolle DD', u'count': 145807},
 {u'_id': u'NESDD', u'count': 79159},
 {u'_id': u'Thomas8122', u'count': 75621},
 {u'_id': u'stw1701', u'count': 57403},
 {u'_id': u'Wurgwitz', u'count': 33881},
 {u'_id': u'iknopf', u'count': 33364},
 {u'_id': u'vaust', u'count': 31138},
 {u'_id': u'ubahnverleih', u'count': 29756},
 {u'_id': u'master', u'count': 28207},
 '...',
 {u'_id': u'freidageorgi', u'count': 1},
 {u'_id': u'kswim', u'count': 1},
 {u'_id': u'itiboi', u'count': 1},
 {u'_id': u'oliverlindner', u'count': 1},
 {u'_id': u'diemirk', u'count': 1},
 {u'_id': u'chkr', u'count': 1},
 {u'_id': u'The King', u'count': 1},
 {u'_id': u'mowsw', u'count': 1},
 {u'_id': u'kicherschleife', u'count': 1},
 {u'_id': u'choess', u'count': 1}]


###Types of Amenities
The attribute ```amenity``` inspired me to do further research in which kind of buildings / objects / facilities are stored in the Open Street Map data in larger quantities in order to do more detailed research on those objects. Especially [Restaurants](#Cuisines-in-Restaurants), [Pubs](#Beers-in-Pubs) and [Churches / Places of Worship](#Religions-in-Places-of-Worship) were investigated further (as can be seen below).

In [12]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_amenities(...):
l = list(project_coll.aggregate([
            {"$match": {"amenity": {"$exists": True}}},
            {"$group": {"_id": "$amenity", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]))
pprint.pprint(l[1:20]+['...'])

[{u'_id': u'parking', u'count': 2689},
 {u'_id': u'bicycle_parking', u'count': 959},
 {u'_id': u'recycling', u'count': 892},
 {u'_id': u'post_box', u'count': 793},
 {u'_id': u'restaurant', u'count': 684},
 {u'_id': u'vending_machine', u'count': 616},
 {u'_id': u'waste_basket', u'count': 536},
 {u'_id': u'fast_food', u'count': 330},
 {u'_id': u'telephone', u'count': 329},
 {u'_id': u'kindergarten', u'count': 322},
 {u'_id': u'school', u'count': 282},
 {u'_id': u'cafe', u'count': 268},
 {u'_id': u'hunting_stand', u'count': 192},
 {u'_id': u'doctors', u'count': 171},
 {u'_id': u'grit_bin', u'count': 158},
 {u'_id': u'pharmacy', u'count': 150},
 {u'_id': u'place_of_worship', u'count': 150},
 {u'_id': u'fountain', u'count': 133},
 {u'_id': u'pub', u'count': 131},
 '...']


###Popular Leisure Activities
The attribute ```leisure``` shows the types of leisure activities one can do in Dresden and inspired me to invesigate more on [popular sports in the city](#Popular-Sports) (```leisure```=```sports_center``` or ```leisure```=```stadium```).

In [15]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_amenities(...):
l = list(project_coll.aggregate([
            {"$match": {"leisure": {"$exists": True}}},
            {"$group": {"_id": "$leisure", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]))
pprint.pprint(l[1:20]+['...'])

[{u'_id': u'pitch', u'count': 575},
 {u'_id': u'park', u'count': 431},
 {u'_id': u'sports_centre', u'count': 193},
 {u'_id': u'garden', u'count': 190},
 {u'_id': u'swimming_pool', u'count': 108},
 {u'_id': u'track', u'count': 48},
 {u'_id': u'dance', u'count': 23},
 {u'_id': u'water_park', u'count': 22},
 {u'_id': u'stadium', u'count': 19},
 {u'_id': u'table_tennis_table', u'count': 17},
 {u'_id': u'marina', u'count': 10},
 {u'_id': u'point_of_interest', u'count': 9},
 {u'_id': u'common', u'count': 8},
 {u'_id': u'slipway', u'count': 8},
 {u'_id': u'horse_riding', u'count': 7},
 {u'_id': u'nature_reserve', u'count': 7},
 {u'_id': u'picnic_table', u'count': 7},
 {u'_id': u'recreation_ground', u'count': 6},
 {u'_id': u'dog_park', u'count': 3},
 '...']


###Religions in Places of Worship
Grouping and sorting by the occurences of the ```religion``` attribute for all ```amenities``` classified as ```place_of_worship``` or ```community_center``` gives us an indication, how prevalent religions are in our city: obviously, ```christian``` is the most prevalent here.

In [17]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_religions(...):
l = list(project_coll.aggregate([
            {"$match": {"amenity":{"$in": ["place_of_worship","community_center"]}}},
            {"$group": {"_id": "$religion", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]))
pprint.pprint(l)

[{u'_id': u'christian', u'count': 140},
 {u'_id': u'muslim', u'count': 3},
 {u'_id': u'buddhist', u'count': 2},
 {u'_id': None, u'count': 2},
 {u'_id': u'jewish', u'count': 2},
 {u'_id': u'multifaith', u'count': 1}]


###Cuisines in Restaurants
We can list the types of ```cuisines``` in restaurants (elements with attribute ```amenity``` matching ```restaurant```) and sort them in decending order. We can notice certain inconsistencies or overlaps in the classifications of this data: e.g., a ```kebab``` cuisine may very well be also classified as an ```arab``` cuisine or may, in fact a sub- or super-classification of this cuisine. One could, e.g., eliminate or cluster together especially occurences of cuisines which are less common, but Without having a formal taxonomy of all cuisines, I decided that is probably best to leave the data as-is in order to not sacrifice preciseness for consistency.

In [18]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_cuisines(...):
l = list(project_coll.aggregate([
            {"$match": {"amenity": "restaurant"}},
            {"$group": {"_id": "$cuisine", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]))
pprint.pprint(l)

[{u'_id': None, u'count': 299},
 {u'_id': u'german', u'count': 66},
 {u'_id': u'regional', u'count': 56},
 {u'_id': u'italian', u'count': 52},
 {u'_id': u'greek', u'count': 28},
 {u'_id': u'asian', u'count': 28},
 {u'_id': u'pizza', u'count': 17},
 {u'_id': u'chinese', u'count': 12},
 {u'_id': u'international', u'count': 11},
 {u'_id': u'indian', u'count': 11},
 {u'_id': u'spanish', u'count': 8},
 {u'_id': u'vietnamese', u'count': 6},
 {u'_id': u'thai', u'count': 5},
 {u'_id': u'steak_house', u'count': 5},
 {u'_id': u'sushi', u'count': 5},
 {u'_id': u'russian', u'count': 4},
 {u'_id': u'american', u'count': 4},
 {u'_id': u'kebab', u'count': 4},
 {u'_id': u'african', u'count': 3},
 {u'_id': u'japanese', u'count': 3},
 {u'_id': u'turkish', u'count': 3},
 {u'_id': u'french', u'count': 3},
 {u'_id': u'mediterranean', u'count': 3},
 {u'_id': u'fish', u'count': 2},
 {u'_id': u'hungarian', u'count': 2},
 {u'_id': u'mexican', u'count': 2},
 {u'_id': u'italian;pizza', u'count': 2},
 {u'_id': u'

###Beers in Pubs
Germans do love their beers and the dataset shows that certain ```pub```s, ```restaurant```s or ```bar```s are sponsored by certain beer brands (often advertised on the pubs entrance). We can analyze the prevalence of beer brands by grouping and sorting by occurence of the attribute ```brewery``` for all the ```amenities``` classified as respective establishment. Most popular are [```Radeberger```](https://en.wikipedia.org/wiki/Radeberger_Brewery), a [very popular](https://www.youtube.com/watch?v=QJ9-euumMzQ) local beer, [```Feldschlösschen```](https://en.wikipedia.org/wiki/Feldschl%C3%B6sschen), a swiss beer and ```Dresdner Felsenkeller```, a very local and niche-sort-of beer.

In [19]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_beers(...):
l = list(project_coll.aggregate([
            {"$match": {"amenity": {"$in":["pub","bar","restaurant"]}}},
            {"$group": {"_id": "$brewery", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]))
pprint.pprint(l)

[{u'_id': None, u'count': 867},
 {u'_id': u'Radeberger', u'count': 10},
 {u'_id': u'Feldschl\xf6\xdfchen', u'count': 3},
 {u'_id': u'Dresdner Felsenkeller', u'count': 3},
 {u'_id': u'Warsteiner', u'count': 2},
 {u'_id': u'Rechenberger', u'count': 2},
 {u'_id': u'Feldschl\xf6\xdfchen;Schwarzer Steiger', u'count': 2},
 {u'_id': u'Einsiedler', u'count': 1},
 {u'_id': u'Eibauer', u'count': 1},
 {u'_id': u'Freiberger', u'count': 1},
 {u'_id': u'Freiberger;Jever;Astra;Lech;Tyskie;B\xf6hmisch Brauhaus',
  u'count': 1},
 {u'_id': u'Kulmbacher', u'count': 1},
 {u'_id': u"Neustadt Helles;Lenin's Hanf", u'count': 1},
 {u'_id': u'Feldschl\xf6\xdfchen;Paulaner;Schwarzer Steiger', u'count': 1},
 {u'_id': u'Radeberger;Eibauer', u'count': 1}]


###Popular Sports
To investigate, which sports are popular, we can group and sort by the (occurence of the) ```sport``` attribute for all elements classified as ```sports_centre``` or ```stadium``` in their ```leisure``` attribute. Unsurprisingly for a german city, we notice that ```9pin``` (bowling) and ```soccer``` are the most popular sports, followed by ```climbing```, an activity very much enjoyed by people in Dresden, presumably because of the close-by sand-stone mountains of the national park [Sächsische Schweiz](http://www.saechsische-schweiz.de/en/saxon-switzerland.html).

In [20]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_sports(...):
l = list(project_coll.aggregate([
            {"$match": {"leisure": {"$in": ["sports_centre","stadium"]}}},
            {"$group": {"_id": "$sport", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]))
pprint.pprint(l)

[{u'_id': None, u'count': 112},
 {u'_id': u'multi', u'count': 55},
 {u'_id': u'9pin', u'count': 5},
 {u'_id': u'soccer', u'count': 5},
 {u'_id': u'climbing', u'count': 4},
 {u'_id': u'gymnastics', u'count': 4},
 {u'_id': u'equestrian', u'count': 2},
 {u'_id': u'athletics', u'count': 2},
 {u'_id': u'tennis', u'count': 2},
 {u'_id': u'judo', u'count': 2},
 {u'_id': u'taekwon_do', u'count': 1},
 {u'_id': u'water_ski', u'count': 1},
 {u'_id': u'table_tennis', u'count': 1},
 {u'_id': u'ice_hockey', u'count': 1},
 {u'_id': u'dancing', u'count': 1},
 {u'_id': u'canoe', u'count': 1},
 {u'_id': u'fitness;matrial_arts', u'count': 1},
 {u'_id': u'aikido', u'count': 1},
 {u'_id': u'Karate', u'count': 1},
 {u'_id': u'taekwondo', u'count': 1},
 {u'_id': u'taekwon-do', u'count': 1},
 {u'_id': u'volleyball', u'count': 1},
 {u'_id': u'yoga', u'count': 1},
 {u'_id': u'fitness', u'count': 1},
 {u'_id': u'shooting', u'count': 1},
 {u'_id': u'skateboard;bmx;unicycle', u'count': 1},
 {u'_id': u'karate', u'c

###Where to Dance in Dresden
I am a passionate social dancer, so a list of dance schools in Dresden should not be abscent from this investigation. We can quickly grab all elements which have the ```leisure``` attribute set to ```dancing```.

In [21]:
from Project.notebook_stub import project_coll
import pprint

# Query used - see function: Project.audit_stats_map.stats_dances(...):
l = list(project_coll.distinct("name", {"leisure": "dance"}))
pprint.pprint(l)

[u'Dancing Joy',
 u'Tanzschule Graf',
 u'Tanzschule Weise',
 u'tres tangos',
 u'Tango im Salon',
 u'Studio24',
 u'La Academia Tango',
 u'Tanzschule Herrmann-Nebl',
 u'Tanzstudio Sandana',
 u'TSC Casino Dresden e.V.',
 u'Tanzschule Nebl',
 u'Tanzclub Galaxy Dresden',
 u'Tanzsportklub RESIDENZ Dresden',
 u'TC Saxonia',
 u'Tanzschule Lax',
 u'Gare de la lune',
 u'Tanzhaus Dresden',
 u'Tanzzentrum Dresden',
 u'Milonga & Wein',
 u'ego-Wohlf\xfchlhaus',
 u'Studio Fischer',
 u'ADTV-Tanzschule Linhart']


##Data Quality
**Note**: Use ```python project.py -q``` to obtain the data from this chapter. See Sample Output in file [```Project/output_project.py_-q.txt```](Project/data/audit_buildings.csv). The script also writes a CSV file to [```Project/data/audit_buildings.csv```](Project/data/audit_buildings.csv), which is also beautified into a [Excel File](Project/data/audit_buildings.xlsx).

###Auditing Street Names
The first idea was to audit the map's street names analogous to how it was done in the Data Wrangling course: check, whether 'weird' street names occur, which do not end on a suffix like ```street``` (in German ```-straße``` or ```Straße```, depending on whether it is a compound word or not), but in a abbreviation like ```str.```. The following code snipped uses a MongoDB query with regular expression which returns all street names <u>not</u> ending with a particular suffix like ```[Ss]traße``` (street), ```[Ww]eg``` (way) etc. This is accomplished by a ["negative lookbehind"](http://www.regular-expressions.info/lookaround.html) regex at the end of the regular expression: ```(?<!...)```, which essentially "looks back" at a position inside the text whether any text preceeding the current position <u>does not</u> match a predefined string. Using multiple of those lookbehinds in sequence allows to match exactly those street names, which do not match any of the predefined suffixes.

In [26]:
from Project.notebook_stub import project_coll

# Query used - see function: Project.audit_quality_map.audit_streets(...):
expectedStreetPattern = \
    u"^.*(?<![Ss]tra\u00dfe)(?<![Ww]eg)(?<![Aa]llee)(?<![Rr]ing)(?<![Bb]erg)" + \
    u"(?<![Pp]ark)(?<![Hh]\u00f6he)(?<![Pp]latz)(?<![Bb]r\u00fccke)(?<![Gg]rund)$"
l = list(project_coll.distinct("name", {
                    "type": "way",
                    "name": {"$regex": expectedStreetPattern}
                }))
# Due to the excessive length of the output, I am not going to print the result. See the file Project/output_project.py_-q.txt

The resulting list was skimmed through and no obvious candidates for cleaning were found. The list was, indeed, quite long. However, the nature of the german language (and how in Germany streetnames work) results in the fact, that there are many small places without a suffix like "street" but "their own thing" (like ```Am Hang``` lit. 'At The Slope', ```Beerenhut``` lit. 'Berry Hat', ```Im Grunde``` lit. 'In The Ground').

###Cross Auditing Street Names with Street Addresses
I did not want to trust the street names of the data set fully yet. Next, I tried figuring out if street names of buildings were consistent with street names of objects in close proximity. Therefore, a JavaScript query is run directly on the database server returning
* all street names (```address.street```) of buildings (elements, where attribute ```buildings``` exists) and which have an assigned position (```pos```) on the map
* with the computed value ```nearby```, an array of ```address.street``` occurences, which is calculated by finding objects close to the current value using the ```$near``` operator

In [30]:
from Project.notebook_stub import project_db

# Query used - see function: Project.audit_quality_map.audit_buildings(...):
buildings_with_streets = project_db.eval('''
            db.osmnodes.ensureIndex({pos:"2dsphere"});
            result = [];
            db.osmnodes.find(
                    {"building": {"$exists": true}, "address.street": {"$exists": true}, "pos": {"$exists": true}},
                    {"address.street": "", "pos": ""}
                ).forEach(function(val, idx) {
                    val.nearby = db.osmnodes.distinct("address.street",
                            {"_id": {"$ne": val._id}, "pos": {"$near": {"$geometry": {"type": "Point", "coordinates": val.pos}, "$maxDistance": 50, "$minDistance": 0}}}
                        );
                    result.push(val);
                })
            return result;
        ''')

# Due to the excessive length of the output, I am not going to print the result. See the file Project/output_project.py_-q.txt

The resulting objects are then iterated through and the best and worst fitting street name are identified each using the [Levenshtein distance](http://stackoverflow.com/questions/18134437/where-can-the-documentation-for-python-levenshtein-be-found-online). For each object, a row is created in a DataFrame which is subsequently exported to a csv file [Project/data/audit_buildings.csv](Project/data/audit_buildings.csv) that was manually beautified into an [Excel File](Project/data/audit_buildings.xlsx). 

![Screenshot of the Excel File](audit_buildings.png)

As can be seen, street names of nearby objects mostly match those of the building itself (Levenshtein distance is zero). If they deviate greatly, they are totally different street names. There seem to be no cases of "typos" where a small Levenshtein distance of one to three is responsible for a different street name.

### Auditing Phone Numbers
I noticed early on, that the phone numbers of Point-of-Interests are scattered over different attributes (```address.phone```, ```phone``` and ```mobile_phone```). Also, phone numbers can come in different styles of formating (like ```+49 351 123 45``` vs. ```0049-351-12345```). To approach this cleaning task, I retrieved a list of all phone numbers. With the goal in mind to later store the normalized phone number back into the attribute ```phone```, which was most widely used in the data set, I retrieved the value from this parameter first, and only if it was empty, I used the parameters ```mobile_phone``` or ```address.phone```. The result of this query was processed further during the cleaning stage.

In [33]:
from Project.notebook_stub import project_coll

# Query used - see function: Project.audit_quality_map.audit_phone_numbers(...):
l = project_coll.aggregate([
        {"$match": {"$or": [
            {"phone": {"$exists": True}},
            {"mobile_phone": {"$exists": True}},
            {"address.phone": {"$exists": True}}
          ]}},
        {"$project": {
            "_id": 1,
            "phone": {"$ifNull": ["$phone", {"$ifNull": ["$mobile_phone", "$address.phone"]}]}
          }}
      ])

# Due to the excessive length of the output, I am not going to print the result. See the file Project/output_project.py_-q.txt

##Cleaning Phone Numbers
**Note:** The information gained in this chapter can be queried using the project script with the ```-C``` option which runs the cleaning script in debug mode. See Sample Output in file ```Project/output_project.py_-C.txt```. The script also writes a CSV file to [```Project/data/clean_phones.csv```](Project/data/clean_phones.csv), which is also beautified into a [Excel File](Project/data/clean_phones.xlsx).

Cleaning the phone numbers involves:
* unifying the different phone attributes (```phone```, ```address.phone``` and ```mobile_phone```) - this is already taken care by extracting the phone numbers during the audit stage
* if possible, canonicalizing the phone number notations by parsing them using a regular expression:

```python
phone_regex = re.compile(ur'^(\(?([\+|\*]|00) *(?P<country>[1-9][0-9]*)\)?)?' +  # country code
                         ur'[ \/\-\.]*\(?0?\)?[ \/\-\.]*' +  # separator
                         ur'(\(0?(?P<area1>[1-9][0-9 ]*)\)|0?(?P<area2>[1-9][0-9]*))?' +  # area code
                         ur'[ \/\-\.]*' +  # separator
                         ur'(?P<number>([0-9]+ *[\/\-.]? *)*)$', # number
                         re.UNICODE)
```

This regular expression was carefully rasped on the [regex101.com online tool](https://regex101.com/) by importing the phone numbers as test dataset.

![A screenshot about regex101.com](clean_regex101.png)

The regular expression is resilient to various separators ("```/```", "```-```", " ", "```(0)```") and bracket notation of phone numbers. It is not resilient for some unicode characters or written lists of phone numbers which are designed to be interpreted by humans (using separators like "```,```", "```/-```" or "```oder```" lit. or). During the cleaning stage, an output is written which phone numbers could not be parsed. This contains only a tiny fraction of phone numbers (9 or 0.5%) which would be easily cleanable by hand.

```
The following objects couldn't be parsed:
                                                 normalized
55f57294b1c8a72c34523897           +49 35207 81429 or 81469
55f57299b1c8a72c345272cd  +49 351 8386837, +49 176 67032256
55f572c2b1c8a72c34546689                      0351 4810426
55f572c3b1c8a72c34546829         +49 351 8902284 or 2525375
55f572fdb1c8a72c34574963   +49 351 4706625, +49 351 0350602
55f573bdb1c8a72c3460bdb3                +49 351 87?44?44?00
55f573bdb1c8a72c3460c066         0162 2648953, 0162 2439168
55f573edb1c8a72c346304b1           03512038973, 03512015831
55f5740eb1c8a72c34649008                0351 4455193 / -118
```

If the phone number was parsable, the country code, area code and rest of the phone number are separated and subsequently strung together to a canonical form using the following guidelines:
* The country code (```country```) is preceded with a "+"
* The area code (```area1``` or ```area2``` - whichever is set) is preceded with a "0" if the country code is not present
* Country code, area code and rest of the number (```number```) are concatenated by " " as a separator

The data to be transformed is stored into a Pandas Dataframe. By using the option ```-C``` instead of ```-c``` the execution of the transformation can be surpressed and the Dataframe instead be written to a [CSV file](Project/data/clean_phones.csv) which might be further beautified into an [Excel File](Project/data/clean_phones.xlsx) in order to test or debug the transformation before writing it to the database with the ```-c``` option.

![A screenshot of the Excel file](clean_phones.png)

##Conclusion and Further Ideas
Overall, the data set of Dresden is pretty neat and tidy. Compared to other, huger cities (e.g., in India), it dawned me when I looked at the Udacity forums, that I had an easier job. There are certainly some other ideas one could venture into:
* The users might be analyzed further: Why are so many nodes (many thousands) created by so few users? Are bots at work?
* Auditing for completeness would be another challenge: Here, we would need another data set as a "gold standard". One could parse several websites for Dresdens street names and test, whether they are present in the data set.
* The report also hints at the possibility to formalize the taxonomy of restaurant cuisines and use a more strict classification method of cuisines. However, areas of art (be it culinary, musical or otherwise) sometimes resist strict classification.
* One could, of course, feed the normalized phone data back into Open Street Map by either using a Web Service or using the XML format.

##References
- [Project Rubric](https://docs.google.com/document/d/1TpfNxDzUjhibq9Qb8cOQHtlvZUelft-W0fb7pCTTyYE/pub)
- [Sample Project](https://docs.google.com/document/d/1F0Vs14oNEs2idFJR3C_OPxwS6L0HPliOii-QpbmrMo4/pub)
- [Lesson & Problem Set 6 of Udacities Data Wrangling with OpenDB Class](https://www.udacity.com/course/viewer#!/c-ud032/l-768058569)
- [Project Evaluation & Submission](https://www.udacity.com/course/viewer#!/c-nd002/l-3168208620/m-3189488621)
- [Python CSV Reader Documentation](https://docs.python.org/2/library/csv.html)
- [Python ElementTree Documentation](https://docs.python.org/2/library/xml.etree.elementtree.html)
- [MongoDB Aggregation Framework Operators](http://docs.mongodb.org/manual/reference/operator/aggregation/project/#pipe._S_project)
- [MongoDB: Indexes](http://docs.mongodb.org/manual/indexes/)
- [Regex Lookarounds](http://www.regular-expressions.info/lookaround.html)
- [MongoDB University](https://university.mongodb.com/)
- [BZip2 Module](https://docs.python.org/2/library/bz2.html)
- [MapZen Metro Extracts](https://mapzen.com/data/metro-extracts)
- [MongoDB Extended JSON](http://docs.mongodb.org/master/reference/mongodb-extended-json/)
- [Retrieving URLs](http://stackoverflow.com/questions/22676/how-do-i-download-a-file-over-http-using-python)
- [Using the Levenshtein distance](http://stackoverflow.com/questions/18134437/where-can-the-documentation-for-python-levenshtein-be-found-online)

